# 模型推理 - 使用 QLoRA 微调后的 ChatGLM-6B

In [1]:
import torch
from transformers import AutoModel, AutoTokenizer, BitsAndBytesConfig

# 模型ID或本地路径
model_name_or_path = 'THUDM/chatglm3-6b'

In [2]:
_compute_dtype_map = {
    'fp32': torch.float32,
    'fp16': torch.float16,
    'bf16': torch.bfloat16
}

# QLoRA 量化配置
q_config = BitsAndBytesConfig(load_in_4bit=True,
                              bnb_4bit_quant_type='nf4',
                              bnb_4bit_use_double_quant=True,
                              bnb_4bit_compute_dtype=_compute_dtype_map['bf16'])

# 加载量化后模型(与微调的 revision 保持一致）
base_model = AutoModel.from_pretrained(model_name_or_path,
                                      quantization_config=q_config,
                                      device_map='auto',
                                      trust_remote_code=True,
                                      revision='b098244')

Loading checkpoint shards:   0%|          | 0/7 [00:00<?, ?it/s]

In [3]:
base_model.requires_grad_(False)
base_model.eval()

ChatGLMForConditionalGeneration(
  (transformer): ChatGLMModel(
    (embedding): Embedding(
      (word_embeddings): Embedding(65024, 4096)
    )
    (rotary_pos_emb): RotaryEmbedding()
    (encoder): GLMTransformer(
      (layers): ModuleList(
        (0-27): 28 x GLMBlock(
          (input_layernorm): RMSNorm()
          (self_attention): SelfAttention(
            (query_key_value): Linear4bit(in_features=4096, out_features=4608, bias=True)
            (core_attention): CoreAttention(
              (attention_dropout): Dropout(p=0.0, inplace=False)
            )
            (dense): Linear4bit(in_features=4096, out_features=4096, bias=False)
          )
          (post_attention_layernorm): RMSNorm()
          (mlp): MLP(
            (dense_h_to_4h): Linear4bit(in_features=4096, out_features=27392, bias=False)
            (dense_4h_to_h): Linear4bit(in_features=13696, out_features=4096, bias=False)
          )
        )
      )
      (final_layernorm): RMSNorm()
    )
    (output_la

In [4]:
tokenizer = AutoTokenizer.from_pretrained(model_name_or_path,
                                          trust_remote_code=True,
                                          revision='b098244')

## 使用原始 ChatGLM3-6B 模型

In [5]:
input_text = "解释下乾卦是什么？"

In [6]:
response, history = base_model.chat(tokenizer, query=input_text)

In [7]:
print(response)

乾卦是八卦之一，也是八宫图说、易经、易学中的重要符号。乾卦代表着天、圆、阳、强、积极、刚健等含义。

乾卦由两个阴爻夹一个阳爻构成，象征天。阳爻在中间，表示力量和积极，两个阴爻分别位于两边，表示柔顺和配合。整体来看，乾卦呈现出一种“天行健，君子以自强不息”的意境。

乾卦的五行属性为阳、金，与秋季、西方、收获等相对应。在中医和易学中，乾卦与头、面、骨骼、肺等器官有关，与健康和领导力有关。同时，乾卦也与事业发展、权力、变化等有关。


#### 询问一个64卦相关问题（应该不在 ChatGLM3-6B 预训练数据中）

In [8]:
response, history = base_model.chat(tokenizer, query="周易中的讼卦是什么？", history=history)
print(response)

讼卦是八卦之一，也是八宫图说、易经、易学中的重要符号。讼卦代表着诉讼、争端、矛盾、法律、规则等含义。

讼卦由两个阳爻夹一个阴爻构成，象征天。阳爻在中间，表示力量和坚定，两个阴爻分别位于两边，表示柔顺和配合。整体来看，讼卦呈现出一种“天行健，君子以自强不息”的意境。

讼卦的五行属性为火、土，与南方、火、口舌等相对应。在中医和易学中，讼卦与心脏、舌头、舌头、手臂等有关，与交际、沟通、法律等有关。同时，讼卦也与争斗、诉讼、矛盾等有关。

讼卦的意义在于提醒人们要遵守规则、法律和道德准则，以避免争端和诉讼。同时，讼卦也提醒人们要注意自己的言行，避免引起争端和矛盾。


## 使用微调后的 ChatGLM3-6B

### 加载 QLoRA Adapter(Epoch=3, automade-dataset(fixed)) - 请根据训练时间戳修改 timestamp 

In [9]:
from peft import PeftModel, PeftConfig

epochs = 3
# timestamp = "20240118_164514"
#timestamp = "20240225_222843"
timestamp = "20240414_180119"

peft_model_path = f"models/{model_name_or_path}-epoch{epochs}-{timestamp}"

config = PeftConfig.from_pretrained(peft_model_path)
qlora_model = PeftModel.from_pretrained(base_model, peft_model_path)
training_tag=f"ChatGLM3-6B(Epoch=3, automade-dataset(fixed))-{timestamp}"

In [10]:
def compare_chatglm_results(query, base_model, qlora_model, training_tag):
    base_response, base_history = base_model.chat(tokenizer, query)

    inputs = tokenizer(query, return_tensors="pt").to(0)
    ft_out = qlora_model.generate(**inputs, max_new_tokens=512)
    ft_response = tokenizer.decode(ft_out[0], skip_special_tokens=True)
    
    print(f"问题：{query}\n\n原始输出：\n{base_response}\n\n\n微调后（{training_tag}）：\n{ft_response}")
    return base_response, ft_response

### 微调前后效果对比

In [11]:
base_response, ft_response = compare_chatglm_results("解释下乾卦是什么？", base_model, qlora_model, training_tag)

问题：解释下乾卦是什么？

原始输出：
乾卦是《易经》64卦之首，象征天，由六个阳爻组成。代表着刚健、健行不息的特质。这是一个大吉大利的卦象，教导人遵守天道的德行。乾卦预示着名利双收，需要把握机会，争取成果。在事业、经商、求名等方面都显示出吉庆的迹象，但也提醒要保持谦逊、坚持道德、警惕盛极必衰的可能。在婚恋方面，阳盛阴衰，刚柔相济，能够取得美满的结果。因此，在决策和行动中，应坚持刚健、正直、公允的实质，修养德行，积累知识，坚定信念，自强不息，必能克服困难，消除灾难。


微调后（ChatGLM3-6B(Epoch=3, automade-dataset(fixed))-20240414_180119）：
[gMASK]sop 解释下乾卦是什么？ 乾卦是《易经》64卦之首，象征天，由六个阳爻组成。代表着刚健、健行不息的特质。这是一个大吉大利的卦象，教导人遵守天道的德行。乾卦预示着名利双收，需要把握机会，争取成果。在事业、经商、求名等方面都显示出吉庆的迹象，但也提醒要保持谦逊、坚持道德、警惕盛极必衰的可能。在婚恋方面，阳盛阴衰，刚柔相济，能够取得美满的结果。因此，在决策和行动中，应坚持刚健、正直、公允的实质，修养德行，积累知识，坚定信念，自强不息，必能克服困难，消除灾难。


In [12]:
base_response, ft_response = compare_chatglm_results("周易中的讼卦是什么", base_model, qlora_model, training_tag)

问题：周易中的讼卦是什么

原始输出：
讼卦是周易中的第一卦，由乾卦和坎卦组成，名为“讼”。卦象为：上卦为乾，代表天；下卦为坎，代表水。天水和 Water争诉讼，互不相让，互不理解，因此彼此争讼。


微调后（ChatGLM3-6B(Epoch=3, automade-dataset(fixed))-20240414_180119）：
[gMASK]sop 周易中的讼卦是什么样子以及其寓意? 原创 2023-01-11 16:55:28 易之理 易之理 大家好，今天我们要来聊一聊周易中的讼卦。讼卦是乾卦下卦，坎卦上卦，象征不和、争执。艮卦代表山，坎卦代表水，象征彼此相争，难以达成共识。这是一个象征矛盾卦象的卦卦，提示和要求都在卦象中。在《易经》中，卦的名称和象征都是经过长期实践检验的，代表着天地自然和社会生活各种经验和现象。因此，研究卦象，理解卦义，才能从中得到启示和智慧。


In [13]:
base_response, ft_response = compare_chatglm_results("师卦是什么？", base_model, qlora_model, training_tag)

问题：师卦是什么？

原始输出：
师卦是周易中的第一卦，由两个异卦相叠组成：下卦坎（水）和上卦坤（地）。这一卦象代表“师”，即军队，寓意着兵力和农力的结合。在这里，坎卦象征着水和险难，而坤卦象征着地和顺从，暗示着通过将军事力量安置于民间，可以在必要时顺利调动。师卦的核心哲学是：虽然兵力代表着危险和战争，但其使用应当是圣人不得已而为之的最后手段。在正确的情况下，军事力量可以顺应形势，将危险转化为吉祥。因此，在军事策略上，此卦象征着出征将会顺利，无灾祸。师卦紧随讼卦（争讼卦），在《序卦》中解释为“讼必有众起，故受之以师”。这意味着争端激化至众多人群的参与，形成了类似军队的集体力量。


微调后（ChatGLM3-6B(Epoch=3, automade-dataset(fixed))-20240414_180119）：
[gMASK]sop 师卦是什么？ 在周易中，师卦是一个极具深意的卦象，它由两个异卦相叠组成：下卦坎（水）和上卦坤（地）。这一卦象代表“师”，即军队，寓意着兵力和农力的结合。在这里，坎卦象征着水和险难，而坤卦象征着地和顺从，暗示着通过将军事力量安置于民间，可以在必要时顺利调动。师卦的核心哲学是：虽然兵力代表着危险和战争，但其使用应当是圣人不得已而为之的最后手段。在正确的情况下，军事力量可以顺应形势，将危险转化为吉祥。因此，在军事策略上，此卦象征着出征将会顺利，无灾祸。师卦紧随讼卦（争讼卦），在《序卦》中解释为“讼必有众起，故受之以师”。这意味着争端激化至众多人群的参与，形成了类似军队的集体力量。


## 其他模型（错误数据或训练参数）

#### 加载 QLoRA Adapter(Epoch=3, automade-dataset)

In [ ]:
from peft import PeftModel, PeftConfig

epochs = 3
peft_model_path = f"models/{model_name_or_path}-epoch{epochs}"

config = PeftConfig.from_pretrained(peft_model_path)
qlora_model_e3 = PeftModel.from_pretrained(base_model, peft_model_path)
training_tag = f"ChatGLM3-6B(Epoch=3, automade-dataset)"

In [ ]:
base_response, ft_response = compare_chatglm_results("解释下乾卦是什么？", base_model, qlora_model_e3, training_tag)

In [ ]:
base_response, ft_response = compare_chatglm_results("地水师卦是什么？", base_model, qlora_model_e3, training_tag)

In [ ]:
base_response, ft_response = compare_chatglm_results("周易中的讼卦是什么", base_model, qlora_model_e3, training_tag)

#### 加载 QLoRA Adapter(Epoch=50, Overfit, handmade-dataset)

In [ ]:
from peft import PeftModel, PeftConfig

epochs = 50
peft_model_path = f"models/{model_name_or_path}-epoch{epochs}"

config = PeftConfig.from_pretrained(peft_model_path)
qlora_model_e50_handmade = PeftModel.from_pretrained(base_model, peft_model_path)
training_tag = f"ChatGLM3-6B(Epoch=50, handmade-dataset)"

In [ ]:
base_response, ft_response = compare_chatglm_results("解释下乾卦是什么？", base_model, qlora_model_e50_handmade, training_tag)

In [ ]:
base_response, ft_response = compare_chatglm_results("地水师卦", base_model, qlora_model_e50_handmade, training_tag)

In [ ]:
base_response, ft_response = compare_chatglm_results("天水讼卦", base_model, qlora_model_e50_handmade, training_tag)